In [5]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError
from ml_functions import *
from sklearn.metrics import accuracy_score, recall_score
from db_queries import username, password, dsn, dbhostname, service_name, dbtables, querys
from table_functions import *
import mlflow
import mlflow.pyfunc
import xgboost as xgb
from main import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
def combine_final_whole_table(data_, dgm_smallest = 8, dgm_biggest = 10):

    data = data_.copy()
    # usuwanie znaków białych z DMC[MEB_DGM] i DMC_CASTING[MEB_DMC]
    data['MEB_DMC'].dmc_casting = data['MEB_DMC']['dmc_casting'].str.strip()
    data['MEB_DGM'].dmc = data['MEB_DGM']['dmc'].str.strip()

    # usuwanie z meb_dmc wierszy z 'WORKPIECE NIO' w kodzie DMC
    data['MEB_DMC'] = data['MEB_DMC'][~data['MEB_DMC']['dmc'].str.contains('WORKPIECE', case=False, na=False)]

    # wybieranie rekordów dla MEB+ 
    data['MEB_DGM'] = data['MEB_DGM'][(data['MEB_DGM']['nr_dgm'].between(dgm_smallest, dgm_biggest)) & (data['MEB_DGM']['dmc'].apply(lambda x: len(str(x)) == 21))]
    # usunięcie anomalii z MEB_DMC
    data['MEB_DMC'] = data['MEB_DMC'][data['MEB_DMC']['dmc'].str[:3] == '0MH']

    # łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ
    data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
    data['MEB_KO'].drop(columns=['indeks'], inplace=True)
    data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
    data['MEB_KO'].drop(columns=['indeks'], inplace=True)
    data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
    data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)
    data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
    data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

    # łączę tabelę MEB_DMC z tabelą MEB_KO
    data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KO'], on='id_dmc', how='left')
    data['MEB_DMC'].drop(columns=['rn'], inplace=True)

    # łączę tabelę MEB_DMC z tabelą MEB_GROB
    data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_GROB'], on='id_dmc', how='left')
    data['MEB_DMC'].drop(columns=['rn'], inplace=True)

    # łączę tabelę MEB_DMC z tabelą MEB_KS
    data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KS'], on='id_dmc', how='left')
    data['MEB_DMC'].drop(columns=['rn'], inplace=True)

    # przygotowywuję tabelę ONI_CIRCUITS do połączenia
    data['ONI_CIRCUITS'].drop(columns = ['assigment', 'working_mode', 'set_point'], inplace = True)
    oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['flow', 'start_delay', 'temp'])
    oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
    oni_circuits.reset_index(inplace=True)

    final_table = data['MEB_DGM'].copy()
    final_table.drop(columns=['rn'], inplace=True)

    # łączę tabelę MEB_DGM z tabelą MEB_KO_DGM
    final_table = final_table.merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
    final_table.drop(columns=['rn'], inplace=True)

    # łączę z tabelą MEB_DGM
    final_table = final_table.merge(oni_circuits, left_on='id', right_on='id_dmc', how='inner')
    final_table.drop(columns=['id_dmc_y'], inplace=True)
    final_table.rename(columns={'id_dmc_x': 'id_dmc'}, inplace=True)

    # łączę tabelę MEB_DMC z ONI_CIRCUITS
    final_table = final_table.merge(data['MEB_DMC'], left_on='dmc', right_on='dmc_casting', how='left', suffixes=('_DGM', '_DMC'))

    # duplicate_count_oni = final_table['dmc_DMC'].duplicated(keep=False).sum()
    # print(f"Number of rows with the same 'dmc' value: {duplicate_count_oni}")

    #final_table.drop(columns=['nok_strefa_DGM', 'nok_rodzaj_DGM', 'status_ko_DGM', 'kod_pola_DGM', 'rodzaj_uszkodzenia_DGM'], inplace=True)
    # final_table.rename(columns={'nok_strefa_DMC': 'nok_strefa', 'nok_rodzaj_DMC': 'nok_rodzaj', 
    #                             'status_ko_DMC': 'status_ko', 'kod_pola_DMC': 'kod_pola', 
    #                             'rodzaj_uszkodzenia_DMC': 'rodzaj_uszkodzenia'}, inplace=True)
                                
    final_table.drop(index=final_table[(final_table['dmc_DGM'].duplicated(keep=False)) & (~final_table['dmc_casting'].isna())].index, inplace=True)
    final_table.drop(columns = ['part_status'], inplace = True)

    return final_table

In [ ]:
data = read_data_from_database()
whole_table = combine_final_whole_table(data, 9, 10)

In [4]:
print('Save whole table')
save_df_to_csv(whole_table, 'final_whole_table.csv')

Save whole table


In [2]:
whole_table = load_csv('final_whole_table.csv')

File to read:
c:/Users/DLXPMX8/Desktop/Projekt_AI/meb_process_data_analysis/src/.data/final_whole_table.csv


c:\Users\DLXPMX8\Desktop\Projekt_AI\meb_process_data_analysis\src\table_functions.py:229: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(current_dir, '.data', file_name).replace("\\","/"))


In [6]:
whole_table.head()

,id,dmc_DGM,nr_dgm,status,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,temp_pieca,oni_temp_curr_f1,oni_temp_fore_f1,oni_temp_fore_f2,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,data_odlania,id_dmc_DGM,nok_strefa_DGM,nok_rodzaj_DGM,status_ko_DGM,rodzaj_kontroli,product_id,line_id,kod_pola_DGM,rodzaj_uszkodzenia_DGM,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,flow_21,flow_22,flow_23,flow_24,flow_25,flow_26,flow_27,flow_28,start_delay_1,start_delay_2,start_delay_3,start_delay_4,start_delay_5,start_delay_6,start_delay_7,start_delay_8,start_delay_9,start_delay_10,start_delay_11,start_delay_12,start_delay_13,start_delay_14,start_delay_15,start_delay_16,start_delay_17,start_delay_18,start_delay_19,start_delay_20,start_delay_21,start_delay_22,start_delay_23,start_delay_24,start_delay_25,start_delay_26,start_delay_27,start_delay_28,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28,id_dmc_DMC,dmc_DMC,dmc_casting,nok_strefa_DMC,nok_rodzaj_DMC,status_ko_DMC,kod_pola_DMC,rodzaj_uszkodzenia_DMC,part_type,temp_workpiece,temp_hydraulics,pressure_pcf_1,pressure_pcf_2,pressure_pcf_3,cisnienie,przeciek,nrprogramu,temperaturatestu,statusszczelnosc,statusdmc
0,315793,210623155611061147233,10.0,1,2277.0,90.0,45.0,6.30,5.0,282.0,27.0,35.0,677.85,79.8,81.9,-1.0,5713.976,91.37731,90.79861,94.96528,171.2384,130.9028,94.0,2021-06-23 15:56:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11.1,14.5,10.8,14.3,7.8,9.1,0.0,2.5,9.3,9.8,0.0,10.3,11.5,11.6,11.6,11.6,12.6,11.5,10.6,3.7,0.0,11.1,8.5,15.0,11.6,13.5,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,85.9,85.7,85.9,96.2,82.7,79.8,80.7,78.7,81.7,72.6,87.1,91.4,81.9,82.4,79.3,79.0,80.4,78.2,82.6,78.6,74.1,78.5,74.8,80.9,80.5,72.3,86.8,77.6,289563.0,0MH301103DVW42106238970263102S612106231556,210623155611061147233,0.0,0.0,1.0,Brak,Brak,1.0,27.400000,40.200001,101.0,104.0,90.0,1.061,0.000,1.0,31.199999,1.0,1.0
1,315795,210623155710941036846,9.0,1,2220.0,76.0,16.0,5.00,4.0,281.0,0.0,41.0,0.00,71.8,70.1,-1.0,4910.590,77.66204,73.61111,74.30556,381.5393,397.5116,85.0,2021-06-23 15:57:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11.8,15.1,12.1,16.7,7.6,11.6,0.0,0.0,10.1,11.0,0.0,11.6,11.1,13.3,11.5,11.8,12.8,12.8,14.3,4.6,0.0,13.5,9.1,14.3,12.3,15.6,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,20.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,80.3,83.2,92.4,87.5,73.7,72.1,72.8,71.3,84.6,68.2,79.2,83.2,74.4,74.3,71.3,70.3,73.5,75.0,72.1,69.3,67.8,76.6,67.9,72.5,69.0,71.3,85.7,70.0,289614.0,0MH301103DVW42106238976242102S412106231557,210623155710941036846,0.0,0.0,1.0,Brak,Brak,1.0,27.000000,39.500000,99.0,99.0,88.0,1.060,0.006,1.0,30.799999,1.0,1.0
2,315816,210623155811061147234,10.0,1,2277.0,91.0,44.5,6.26,5.0,282.0,27.0,34.0,678.20,79.5,81.7,-1.0,5751.302,66.08796,64.93056,70.83333,165.7986,121.9907,95.0,2021-06-23 15:58:09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11.1,14.5,10.8,14.3,7.8,9.1,0.0,2.7,9.3,9.6,0.0,10.3,11.3,11.1,11.6,11.8,12.6,11.5,10.6,3.7,0.0,11.1,8.5,14.8,11.6,13.5,7.0,15.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,12.0,2.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,2.0,21.0,2.0,2.0,2.0,2.0,2.0,85.9,85.7,85.8,96.8,82.5,79.8,80.5,79.1,81.8,73.1,87.3,91.5,81.4,82.9,79.6,79.2,80.8,77.9,82.4,77.9,74.1,78.9,74.9,81.0,80.2,72.4,86.8,78.2,289562.0,0MH301103DVW42106238970153102S612106231558,210623155811061147234,0.0,0.0,1.0,Brak,Brak,1.0,27.400000,40.200001,101.0,104.0,90.0,1.057,0.004,1.0,31.799999,1.0,1.0
3,315817,210623155911061147235,10.0,1,2275.0,89.0,43.5,6.37,5.0,282.0,27.0,33.0,678.22,79.8,82.0,-1.0,5785.156,71.06481,69.90741,75.75231,164.4097,

In [9]:
whole_table[whole_table['status']==10].tail()

,id,dmc_DGM,nr_dgm,status,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,temp_pieca,oni_temp_curr_f1,oni_temp_fore_f1,oni_temp_fore_f2,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,data_odlania,id_dmc_DGM,nok_strefa_DGM,nok_rodzaj_DGM,status_ko_DGM,rodzaj_kontroli,product_id,line_id,kod_pola_DGM,rodzaj_uszkodzenia_DGM,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,flow_21,flow_22,flow_23,flow_24,flow_25,flow_26,flow_27,flow_28,start_delay_1,start_delay_2,start_delay_3,start_delay_4,start_delay_5,start_delay_6,start_delay_7,start_delay_8,start_delay_9,start_delay_10,start_delay_11,start_delay_12,start_delay_13,start_delay_14,start_delay_15,start_delay_16,start_delay_17,start_delay_18,start_delay_19,start_delay_20,start_delay_21,start_delay_22,start_delay_23,start_delay_24,start_delay_25,start_delay_26,start_delay_27,start_delay_28,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28,id_dmc_DMC,dmc_DMC,dmc_casting,nok_strefa_DMC,nok_rodzaj_DMC,status_ko_DMC,kod_pola_DMC,rodzaj_uszkodzenia_DMC,part_type,temp_workpiece,temp_hydraulics,pressure_pcf_1,pressure_pcf_2,pressure_pcf_3,cisnienie,przeciek,nrprogramu,temperaturatestu,statusszczelnosc,statusdmc
730512,1497844,240126033930911025995,9.0,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25.6,25.6,25.6,1.0,1.0,1.0,1.0,1.0,1.0,406.0,2024-01-26 03:39:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730515,1497865,240126035530911025998,9.0,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25.6,25.6,25.6,1.0,1.0,1.0,1.0,1.0,1.0,767.0,2024-01-26 03:55:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730516,1497946,240126045930911025999,9.0,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25.6,25.6,25.6,1.0,1.0,1.0,1.0,1.0,1.0,3791.0,2024-01-26 04:59:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730517,1497949,240126050130911026000,9.0,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25.6,25.6,25.6,1.0,1.0,1.0,1.0,1.0,1.0,132.0,2024-01-26 05:01:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.

In [28]:
whole_table['data_odlania'] = pd.to_datetime(whole_table['data_odlania'])
filtered_data = whole_table[whole_table['data_odlania'].dt.year >= 2023]

In [29]:
filtered_data.head()

,id,dmc_DGM,nr_dgm,status,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,nachdruck_hub,anguss,temp_pieca,oni_temp_curr_f1,oni_temp_fore_f1,oni_temp_fore_f2,vds_air_pressure,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,data_odlania,id_dmc_DGM,nok_strefa_DGM,nok_rodzaj_DGM,status_ko_DGM,rodzaj_kontroli,product_id,line_id,kod_pola_DGM,rodzaj_uszkodzenia_DGM,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20,flow_21,flow_22,flow_23,flow_24,flow_25,flow_26,flow_27,flow_28,start_delay_1,start_delay_2,start_delay_3,start_delay_4,start_delay_5,start_delay_6,start_delay_7,start_delay_8,start_delay_9,start_delay_10,start_delay_11,start_delay_12,start_delay_13,start_delay_14,start_delay_15,start_delay_16,start_delay_17,start_delay_18,start_delay_19,start_delay_20,start_delay_21,start_delay_22,start_delay_23,start_delay_24,start_delay_25,start_delay_26,start_delay_27,start_delay_28,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28,id_dmc_DMC,dmc_DMC,dmc_casting,nok_strefa_DMC,nok_rodzaj_DMC,status_ko_DMC,kod_pola_DMC,rodzaj_uszkodzenia_DMC,part_type,temp_workpiece,temp_hydraulics,pressure_pcf_1,pressure_pcf_2,pressure_pcf_3,cisnienie,przeciek,nrprogramu,temperaturatestu,statusszczelnosc,statusdmc
425811,867282,230102154230981004292,9.0,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25.6,25.6,25.6,1.000,1.00000,1.00000,1.00000,1.0000,1.0000,32767.0,2023-01-02 15:42:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425812,867284,230102172430981004293,9.0,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25.6,25.6,25.6,1.000,1.00000,1.00000,1.00000,1.0000,1.0000,6096.0,2023-01-02 17:24:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425813,867285,230102172630981004294,9.0,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25.6,25.6,25.6,1.000,1.00000,1.00000,1.00000,1.0000,1.0000,117.0,2023-01-02 17:26:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425814,867286,230102172830981004295,9.0,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,25.6,25.6,25.6,1.000,1.00000,1.00000,1.00000,1.0000,1.0000,118.0,2023-01-02 17:28:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,25.6,

In [13]:
whole_table['status'].value_counts()

status
1     687715
10     11808
5       8340
11      6989
3       6337
7       3132
14      2940
4       2698
8        595
Name: count, dtype: int64

In [14]:
whole_table['status_ko_DGM'].value_counts()

status_ko_DGM
1.0     17113
2.0       371
16.0      222
Name: count, dtype: int64

In [15]:
whole_table['status_ko_DMC'].value_counts()     #status KO - 2 to źle i to się łączy z nok_rodzaj i to po indeksie z rodzaj_uszkodzenia

status_ko_DMC
1.0      631293
2.0       15753
0.0         122
106.0        87
Name: count, dtype: int64

In [24]:
whole_table['rodzaj_uszkodzenia_DMC'].value_counts()

rodzaj_uszkodzenia_DMC
Brak                          631501
Odlewnicze - porowatości       12843
Wymiarowe - obróbka MF           691
Mechaniczne - obróbka MF         661
Mechaniczne - odlewnicze         549
Odlewnicze - wtrącenia           542
Odlewnicze - brak naddatku       428
Wymiarowe - odlewnicze            40
Name: count, dtype: int64

In [21]:
whole_table['nok_rodzaj_DMC'].value_counts() # MEB_KO_RODZAJ i tam po indeksie
# Do zostawienia: 0, 102, 201, 103, 101, 

nok_rodzaj_DMC
0.0      631501
102.0     12843
302.0       691
202.0       661
201.0       549
103.0       542
101.0       428
301.0        40
Name: count, dtype: int64

In [16]:
whole_table['statusdmc'].value_counts()

statusdmc
1.0    119149
2.0       466
0.0        61
Name: count, dtype: int64

In [17]:
whole_table['status_ko_DMC'].value_counts()

status_ko_DMC
1.0      631293
2.0       15753
0.0         122
106.0        87
Name: count, dtype: int64

In [25]:
whole_table['statusszczelnosc'].value_counts()

statusszczelnosc
1.0    654285
2.0      2652
3.0       650
0.0         2
Name: count, dtype: int64